In [1]:
import os
import time
import pandas as pd 
import requests
import urllib3
import json
import sys
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta

pd.options.display.max_colwidth = 1000
pd.set_option('display.max_columns', None)

### Files Yara Rules

In [2]:
file_twitter_dir = r'C:\Users\leona\OneDrive\Desktop\Tesi\Pipeline\Datasets\Pipeline\SocialNetworksPosts\Twitter\TwitterPosts.csv'

In [3]:
df_twitter = pd.read_csv(file_twitter_dir, low_memory = False)
df_twitter = df_twitter.drop(columns=['Unnamed: 0', 'json'])
df_twitter

,username,name,date,text
0,SwiftOnSecurity,SwiftOnSecurity,2023-07-12 19:57:11,RT @KimZetter: CISA says it's been working w/ Microsoft to determine the specific log types that are most valuable and should be made avail…
1,SwiftOnSecurity,SwiftOnSecurity,2023-07-12 19:57:09,"RT @KimZetter: In talking about fact that Microsoft cloud compromise was caught because victim had premium 365 logging, CISA says in press…"
2,thegrugq,thaddeus e. grugq thegrugq@infosec.exchange,2023-07-12 20:19:20,"RT @lukOlejnik: NATO reaffirms that cyberattacks may mean war ""single or cumulative set of malicious cyber activities could reach the level…"
3,thegrugq,thaddeus e. grugq thegrugq@infosec.exchange,2023-07-12 20:08:49,RT @matt_meeta: Every German company history
4,k8em0,Katie🌻Moussouris (she/her) @k8em0.bsky.social,2023-07-12 20:04:54,"RT @wdormann: Remember folks, don't play video games on a system that you care about.\nI made a venn diagram to help. https://t.co/YsR22yxPgI"
...,...,...,...,...
49112,PatrickCMiller,Patrick C Miller is on Mastodon,2023-01-01 00:45:01,The Lawfare Podcast: An Investigation into Russian State Media and Disinformation https://t.co/QnnOLwqXgX
49113,BrianHonan,Brian Ó hEoghanáin (Brian Honan) #BLM He/Him,2023-01-01 00:06:34,To all those still in 2022\n\n2023 is already here in Ireland https://t.co/phwmigYahg
49114,BrianHonan,Brian Ó hEoghanáin (Brian Honan) #BLM He/Him,2023-01-01 00:05:15,Happy New Year everyone
49115,PatrickCMiller,Patrick C Miller is on Mastodon,2023-01-01 00:15:00,The LastPass disclosure of leaked password vaults is being torn apart by security experts https://t.co/mn4OrkuEO3


In [4]:
df_twitter['text'] = df_twitter['text'].fillna(' ')
df_twitter

,username,name,date,text
0,SwiftOnSecurity,SwiftOnSecurity,2023-07-12 19:57:11,RT @KimZetter: CISA says it's been working w/ Microsoft to determine the specific log types that are most valuable and should be made avail…
1,SwiftOnSecurity,SwiftOnSecurity,2023-07-12 19:57:09,"RT @KimZetter: In talking about fact that Microsoft cloud compromise was caught because victim had premium 365 logging, CISA says in press…"
2,thegrugq,thaddeus e. grugq thegrugq@infosec.exchange,2023-07-12 20:19:20,"RT @lukOlejnik: NATO reaffirms that cyberattacks may mean war ""single or cumulative set of malicious cyber activities could reach the level…"
3,thegrugq,thaddeus e. grugq thegrugq@infosec.exchange,2023-07-12 20:08:49,RT @matt_meeta: Every German company history
4,k8em0,Katie🌻Moussouris (she/her) @k8em0.bsky.social,2023-07-12 20:04:54,"RT @wdormann: Remember folks, don't play video games on a system that you care about.\nI made a venn diagram to help. https://t.co/YsR22yxPgI"
...,...,...,...,...
49112,PatrickCMiller,Patrick C Miller is on Mastodon,2023-01-01 00:45:01,The Lawfare Podcast: An Investigation into Russian State Media and Disinformation https://t.co/QnnOLwqXgX
49113,BrianHonan,Brian Ó hEoghanáin (Brian Honan) #BLM He/Him,2023-01-01 00:06:34,To all those still in 2022\n\n2023 is already here in Ireland https://t.co/phwmigYahg
49114,BrianHonan,Brian Ó hEoghanáin (Brian Honan) #BLM He/Him,2023-01-01 00:05:15,Happy New Year everyone
49115,PatrickCMiller,Patrick C Miller is on Mastodon,2023-01-01 00:15:00,The LastPass disclosure of leaked password vaults is being torn apart by security experts https://t.co/mn4OrkuEO3


In [5]:
df_twitter['date'] = pd.to_datetime(df_twitter['date']).dt.strftime('%Y-%m-%d')
df_twitter

,username,name,date,text
0,SwiftOnSecurity,SwiftOnSecurity,2023-07-12,RT @KimZetter: CISA says it's been working w/ Microsoft to determine the specific log types that are most valuable and should be made avail…
1,SwiftOnSecurity,SwiftOnSecurity,2023-07-12,"RT @KimZetter: In talking about fact that Microsoft cloud compromise was caught because victim had premium 365 logging, CISA says in press…"
2,thegrugq,thaddeus e. grugq thegrugq@infosec.exchange,2023-07-12,"RT @lukOlejnik: NATO reaffirms that cyberattacks may mean war ""single or cumulative set of malicious cyber activities could reach the level…"
3,thegrugq,thaddeus e. grugq thegrugq@infosec.exchange,2023-07-12,RT @matt_meeta: Every German company history
4,k8em0,Katie🌻Moussouris (she/her) @k8em0.bsky.social,2023-07-12,"RT @wdormann: Remember folks, don't play video games on a system that you care about.\nI made a venn diagram to help. https://t.co/YsR22yxPgI"
...,...,...,...,...
49112,PatrickCMiller,Patrick C Miller is on Mastodon,2023-01-01,The Lawfare Podcast: An Investigation into Russian State Media and Disinformation https://t.co/QnnOLwqXgX
49113,BrianHonan,Brian Ó hEoghanáin (Brian Honan) #BLM He/Him,2023-01-01,To all those still in 2022\n\n2023 is already here in Ireland https://t.co/phwmigYahg
49114,BrianHonan,Brian Ó hEoghanáin (Brian Honan) #BLM He/Him,2023-01-01,Happy New Year everyone
49115,PatrickCMiller,Patrick C Miller is on Mastodon,2023-01-01,The LastPass disclosure of leaked password vaults is being torn apart by security experts https://t.co/mn4OrkuEO3


### NLP sul Behaviours in ogni file

#### Example of NLP

In [6]:
!python -m spacy download it_core_news_sm


     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     --------------------------------------- 0.0/13.0 MB 262.6 kB/s eta 0:00:50
     --------------------------------------- 0.1/13.0 MB 365.7 kB/s eta 0:00:36
     --------------------------------------- 0.2/13.0 MB 766.6 kB/s eta 0:00:17
      -------------------------------------- 0.2/13.0 MB 892.5 kB/s eta 0:00:15
      --------------------------------------- 0.3/13.0 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.5/13.0 MB 1.4 MB/s eta 0:00:09
     - -------------------------------------- 0.5/13.0 MB 1.4 MB/s eta 0:00:09
     -- ------------------------------------- 0.7/13.0 MB 1.6 MB/s eta 0:00:08
     -- ------------------------------------- 0.8/13.0 MB 1.6 MB/s eta 0:00:08
     -- ------------------------------------- 0.9/13.0 MB 1.8 MB/s et

In [7]:
import spacy

nlp = spacy.load("it_core_news_sm")

frase = "Il gatto mangia il pesce."

doc = nlp(frase)

# Estrai sostantivi e verbi dalla frase
parole_di_interesse = [token.text for token in doc if token.pos_ in ['NOUN', 'VERB']]

# Stampa i risultati
print("Parole di interesse:", parole_di_interesse)

Parole di interesse: ['gatto', 'mangia', 'pesce']


#### Get Words from descriptions

In [8]:
pip install spacy


Note: you may need to restart the kernel to use updated packages.


In [9]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 445.2 kB/s eta 0:00:29
     --------------------------------------- 0.1/12.8 MB 409.6 kB/s eta 0:00:32
     --------------------------------------- 0.1/12.8 MB 778.5 kB/s eta 0:00:17
      --------------------------------------- 0.3/12.8 MB 1.1 MB/s eta 0:00:12
     - -------------------------------------- 0.3/12.8 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.4/12.8 MB 1.4 MB/s eta 0:00:09
     - -------------------------------------- 0.5/12.8 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.6/12.8 MB 1.3 MB/s eta 0:00:10
     -- ------------------------------------- 0.7/12.8 MB 1.4 MB/s eta 0:00:09
     -- ------------------------------------- 0.7/12.8 MB 1.4 MB/s eta

In [10]:
selected_rows = df_twitter.loc[(pd.to_datetime(df_twitter['date']).dt.month >= 6) & (pd.to_datetime(df_twitter['date']).dt.year == 2023) ].reset_index(drop =True)
selected_rows

,username,name,date,text
0,SwiftOnSecurity,SwiftOnSecurity,2023-07-12,RT @KimZetter: CISA says it's been working w/ Microsoft to determine the specific log types that are most valuable and should be made avail…
1,SwiftOnSecurity,SwiftOnSecurity,2023-07-12,"RT @KimZetter: In talking about fact that Microsoft cloud compromise was caught because victim had premium 365 logging, CISA says in press…"
2,thegrugq,thaddeus e. grugq thegrugq@infosec.exchange,2023-07-12,"RT @lukOlejnik: NATO reaffirms that cyberattacks may mean war ""single or cumulative set of malicious cyber activities could reach the level…"
3,thegrugq,thaddeus e. grugq thegrugq@infosec.exchange,2023-07-12,RT @matt_meeta: Every German company history
4,k8em0,Katie🌻Moussouris (she/her) @k8em0.bsky.social,2023-07-12,"RT @wdormann: Remember folks, don't play video games on a system that you care about.\nI made a venn diagram to help. https://t.co/YsR22yxPgI"
...,...,...,...,...
15438,CVEnew,CVE,2023-06-01,CVE-2023-2985 A use after free flaw was found in hfsplus_put_super in fs/hfsplus/super.c in the Linux Kernel. This… https://t.co/Ga3aQXZbyC
15439,CVEnew,CVE,2023-06-01,CVE-2023-2977 A vulnerbility was found in OpenSC. This security flaw cause a buffer overrun vulnerability in pkcs15… https://t.co/8rA1oXazvN
15440,runasand,Runa Sandvik,2023-06-01,@TonyaJoRiley Will they accept cats?
15441,runasand,Runa Sandvik,2023-06-01,@riskybusiness Audience feedback is the best.


In [11]:
from tqdm import tqdm

def create_words_df(df):
    nlp = spacy.load("en_core_web_sm")
    combined_texts = df['text'] 
    
    data_list = []
    
    for i, text in tqdm(enumerate(combined_texts), total=len(combined_texts), desc="Processing documents"):
        doc = nlp(text)
        words_interesting = [token.text for token in doc if token.pos_ in ['NOUN', 'VERB']]
        
        for word in words_interesting:
            word_type = 'noun' if nlp(word)[0].pos_ == 'NOUN' else 'verb'
            data_list.append({'word': word, 'type': word_type, 'index': i})
            
    dataframe = pd.DataFrame(data_list).reset_index()
    
    return dataframe


In [12]:
words_df = create_words_df(selected_rows)

Processing documents: 100%|██████████| 15443/15443 [10:34<00:00, 24.34it/s]


In [13]:
words_df

,level_0,word,type,index
0,0,says,verb,0
1,1,working,verb,0
2,2,w/,verb,0
3,3,determine,verb,0
4,4,log,verb,0
...,...,...,...,...
80852,80852,feedback,noun,15441
80853,80853,Research,noun,15442
80854,80854,Employees,noun,15442
80855,80855,Want,verb,15442


In [14]:
words_df.to_csv(r'C:\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\SocialNetworksPosts\Twitter\PostsAnalysis.csv')

In [15]:
merged_words_df = pd.merge(selected_rows, words_df, left_index=True, right_on='index')
merged_words_df = merged_words_df.drop(columns=['index'])

merged_words_df.head()


,username,name,date,text,level_0,word,type
0,SwiftOnSecurity,SwiftOnSecurity,2023-07-12,RT @KimZetter: CISA says it's been working w/ Microsoft to determine the specific log types that are most valuable and should be made avail…,0,says,verb
1,SwiftOnSecurity,SwiftOnSecurity,2023-07-12,RT @KimZetter: CISA says it's been working w/ Microsoft to determine the specific log types that are most valuable and should be made avail…,1,working,verb
2,SwiftOnSecurity,SwiftOnSecurity,2023-07-12,RT @KimZetter: CISA says it's been working w/ Microsoft to determine the specific log types that are most valuable and should be made avail…,2,w/,verb
3,SwiftOnSecurity,SwiftOnSecurity,2023-07-12,RT @KimZetter: CISA says it's been working w/ Microsoft to determine the specific log types that are most valuable and should be made avail…,3,determine,verb
4,SwiftOnSecurity,SwiftOnSecurity,2023-07-12,RT @KimZetter: CISA says it's been working w/ Microsoft to determine the specific log types that are most valuable and should be made avail…,4,log,verb


In [16]:
merged_words_df.to_csv(r'C:\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\SocialNetworksPosts\Twitter\PostsAnalysis.csv')